In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Vehicle_Url'].tolist()

总数量：500


['https://www.1aauto.com/catalog/product/fit?skuId=1176156&pathName=%2Fhonda-accord-acura-cl-rl-tl-tsx-front-driver-and-passenger-side-2-piece-brake-caliper-set-trq-cla34797%2Fi%2F1abcs00162',
 'https://www.1aauto.com/catalog/product/fit?skuId=1152814&pathName=%2Fchevrolet-gmc-cadillac-hummer-driver-and-passenger-side-2-piece-brake-caliper-set-trq-cla48549%2Fi%2F1abcs00016',
 'https://www.1aauto.com/catalog/product/fit?skuId=1169650&pathName=%2Ftoyota-4runner-fj-cruiser-tacoma-front-driver-and-passenger-side-2-piece-brake-caliper-set-trq-cla35380%2Fi%2F1abcs00107',
 'https://www.1aauto.com/catalog/product/fit?skuId=1152833&pathName=%2Fford-f150-rear-driver-and-passenger-side-2-piece-brake-caliper-set-trq-cla48569%2Fi%2F1abcs00036',
 'https://www.1aauto.com/catalog/product/fit?skuId=1152841&pathName=%2Fhonda-accord-acura-tsx-rear-driver-and-passenger-side-2-piece-brake-caliper-set-trq-cla48581%2Fi%2F1abcs00048',
 'https://www.1aauto.com/catalog/product/fit?skuId=1188171&pathName=%2F1965

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get('https://www.1aauto.com/catalog/product/fit?skuId=' + input_.loc[a, 'Product_Id'] + '&pathName=' + input_.loc[a, 'Url'].replace('https://www.1aauto.com', '').replace('/', '%2F'),
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<div id="product-fit-tab">\n\n    <hr class="hr">\n    <div id="available-makes" class="available-makes">\n        \n        <h2>Fits these Makes</h2>\n        <div id="available-makes-list" class="available-makes-list expanderContainer">\n            <div id="fit-makes-filter" class="flex-box fit-makes-filter filter expanderContainerWrapper "\n                 data-filter-target=".fitTable">\n                \n                    <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="Acura">Acura\n                    </button>\n                \n                    <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="Honda">Honda\n                    </button>\n                \n            </div>\n            <div class="expander-container-button">\n                <button class="expanderSwitch closed expander-switch btn btn-link d-none"\n                        data-expander-container="#fit-makes-filter">view more\n       

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div id="product-fit-tab">
   <hr class="hr"/>
   <div class="available-makes" id="available-makes">
    <h2>
     Fits these Makes
    </h2>
    <div class="available-makes-list expanderContainer" id="available-makes-list">
     <div class="flex-box fit-makes-filter filter expanderContainerWrapper" data-filter-target=".fitTable" id="fit-makes-filter">
      <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="Acura">
       Acura
      </button>
      <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="Honda">
       Honda
      </button>
     </div>
     <div class="expander-container-button">
      <button class="expanderSwitch closed expander-switch btn btn-link d-none" data-expander-container="#fit-makes-filter">
       view more
      </button>
     </div>
    </div>
   </div>
   <div class="vehicle-fit cf">
    <div class="fitTable-area">
     <input class="isTableSorted" type="hidden" value="false"/

In [8]:
list_tr = html.xpath('//tr[@class="sortable_row filter-field"]')

list_tr

[<Element tr at 0x127997a40>,
 <Element tr at 0x127a50240>,
 <Element tr at 0x127a50640>,
 <Element tr at 0x127a506c0>,
 <Element tr at 0x127a502c0>,
 <Element tr at 0x127a508c0>,
 <Element tr at 0x127a50880>,
 <Element tr at 0x127a50a40>,
 <Element tr at 0x127a50980>,
 <Element tr at 0x127a50480>,
 <Element tr at 0x127a50ac0>,
 <Element tr at 0x127a50a80>,
 <Element tr at 0x127a50b80>]

In [9]:
dict_vehicle = {}
for tr in list_tr:
    make = tr.xpath('./td[2]/text()')[0].strip()
    model = tr.xpath('./td[3]/text()')[0].strip()
    year_start = int(tr.xpath('./td[1]/text()')[0].split('-')[0].strip())
    year_end = int(tr.xpath('./td[1]/text()')[0].split('-')[-1].strip())
    
    if make not in dict_vehicle:
        dict_vehicle[make] = {model: [year_start, year_end]}
    else:
        if model not in dict_vehicle[make]:
            dict_vehicle[make][model] = [year_start, year_end]
        else:
            dict_vehicle[make][model].append(year_start)
            dict_vehicle[make][model].append(year_end)
                
    dict_vehicle[make][model] = list(set(dict_vehicle[make][model]))
    dict_vehicle[make][model].sort()
                    
dict_vehicle

{'Acura': {'CL': [2001, 2003],
  'RL': [1999, 2004],
  'TL': [1999, 2003, 2004, 2006, 2007, 2008],
  'TSX': [2004, 2014]},
 'Honda': {'Accord': [2003, 2004, 2005, 2006, 2007, 2008, 2012]}}

In [10]:
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        year_max = max(dict_vehicle[make][model])
        year_min = min(dict_vehicle[make][model])
        if year_max == year_min:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
        else:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

vehicle = '\n'.join(list_vehicle)

print(vehicle)

Acura CL 2001-2003
Acura RL 1999-2004
Acura TL 1999-2008
Acura TSX 2004-2014
Honda Accord 2003-2012


In [11]:
oe = ';'.join(sorted(list(set([oe.strip() for oe in html.xpath('//table[@id="interchange-codes-table"]/tbody/tr/td[2]/text()')]))))

oe

'077-1535S;077-1536S;10-5032;10-5033;10-C5032;10-C5033;10042;10047;13020738;13020739;141.40033;141.40034;18FR1843;18FR1843C;18FR1844;18FR1844C;18R1843;18R1844;19-2584;19-2585;19B2584;19B2585;22-00933L;22-00933R;331-59029;45018S0KA01RM;45018SDPA00;45018SDPA01;45018SEAJ02;45019S0KA01RM;45019SDPA00;45019SDPA01;45019SEAJ02;99-00933L-C;99-00933R-C;AA1192584;AA1192585;C40033;C40034;ER1632;ER1632K;ER1633;ER1633K;ER1656;FRC11119;FRC11119C;FRC11119N;FRC11120;FRC11120C;FRC11120N;N1040231312CAR;N1040231313CAR;R47225;R47226;SL19722;SL19723;SLB19722;SLB19723'

In [12]:
df_temp = pd.DataFrame([{'Product_Id': input_.loc[a, 'Product_Id'],
                         'Vehicle': vehicle,
                         'OE': oe}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Product_Id': input_.loc[a, 'Product_Id'],
                             'Url': input_.loc[a, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [15]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Product_Id'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,Product_Id,Vehicle,OE
0,1176156,Acura CL 2001-2003\nAcura RL 1999-2004\nAcura ...,077-1535S;077-1536S;10-5032;10-5033;10-C5032;1...


In [16]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Product_Id'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
